# Indexing Reward Unit Test
Performing verification of the indexing rewards mechanism by comparing the event data to the results of cadCAD model.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')

stakeDelegateds: 605 events
stakeDelegatedLockeds: 375 events
stakeDelegatedWithdrawns: 326 events
allocationCloseds: 47 events
allocationCollecteds: 16 events
stakeDepositeds: 25 events
rewardsAssigneds: 47 events
delegationParametersUpdateds: 6 events
TOTAL NUMBER OF EVENTS: 1529
Set SIMULATION_TIME_STEPS in config.py to 1529



Configure environment

In [2]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)


In [3]:

print("EXPECTED TRUE INDEXING REWARDS FROM REWARDS ASSIGNED EVENTS:")
is_first = True
for timestep, indexing_fee_events_list in rewards_assigned_events.items():
    if is_first:
        is_first = False
        pass
    else:        
        if indexing_fee_events_list is None:
            indexing_fee_amt = 0
        else:
            indexing_fee_amt = sum([e['amount'] for e in indexing_fee_events_list])        
        print(indexing_fee_amt)
        # print(f"{timestep}, {stake_deposited_event[0]['tokens']}")
    


EXPECTED TRUE INDEXING REWARDS FROM REWARDS ASSIGNED EVENTS:
78681.945634908142661532
37810.500232191486929607
26381.732043325393053999
25494.839303727812415387
26234.529693224212529763
49891.902745402016057010
49891.817848670854079448
66214.393501266041029992
31254.043729741473371800
98402.206621725694544502
217207.865614270991850845
265654.897596226056278932
213941.342001342071645712
194137.522791957485390532
91859.973399058188810578
135410.384627951762628518
237821.675242826595473000
173947.355266842636556792
303845.448615810697843104
151930.725822086935104000
2792.339036562857078894
22918.683845921654696336
23950.058834745539489506
31780.514450169181908583
32679.131508368940869594
12373.914467045118773128
10408.072875288181882076
10238.437833754494711791
10933.286787831162710644
17868.796741382750311832
49939.430091121812419000
73035.525482052350460548
37547.801612908386083360
60965.349706255285353420
70875.472929369293578960
74148.542616902969932048
72239.677999385421190800
75240.

In [4]:
print("MODELED RESULTS")
# TODO: weave in indexer_revenue_cut events
# indexer #1
# indexer_revenue_cut = 0.89
# indexer #2
indexer_revenue_cut = Decimal(0.8)
# print(df.iloc[timestep])
is_first = True
indexing_rewards_modeled = {}
for timestep, stake_deposited_event in rewards_assigned_events.items():
    # back indexing rewards out from increase in pool_delegated_stake / 0.11 * 0.89
    if is_first:
        is_first = False
        pass
    else:       
        indexing_rewards =( df.iloc[timestep-1]['pool_delegated_stake'] - df.iloc[timestep-2]['pool_delegated_stake']) / (1 - indexer_revenue_cut) * (indexer_revenue_cut)
        indexing_rewards_modeled[timestep] = indexing_rewards
        print(f"{timestep}, {indexing_rewards=}")
        


MODELED RESULTS
85, indexing_rewards=Decimal('62945.55650792653160012713323')
163, indexing_rewards=Decimal('30248.40018575319793930318607')
167, indexing_rewards=Decimal('21105.38563466032030112046880')
170, indexing_rewards=Decimal('20395.87144298225559330112082')
173, indexing_rewards=Decimal('20987.62375457937584904618113')
293, indexing_rewards=Decimal('39913.52219632162392383583406')
296, indexing_rewards=Decimal('39913.45427893669434176738320')
372, indexing_rewards=Decimal('52971.51480101284752654244534')
405, indexing_rewards=Decimal('25003.23498379318563723179228')
462, indexing_rewards=Decimal('78721.76529738057748528069307')
543, indexing_rewards=Decimal('173766.2924914168417105107069')
633, indexing_rewards=Decimal('212523.9180769809040103823831')
775, indexing_rewards=Decimal('171153.0736010737048210903618')
866, indexing_rewards=Decimal('155310.0182335660314196151495')
945, indexing_rewards=Decimal('73487.97871924657144547390181')
1022, indexing_rewards=Decimal('108328.3

In [9]:
print("UNITTEST RESULTS")
is_first = True
# print(rewards_assigned_events)
cntExact = 0
cntReallyClose = 0
cntClose = 0
cnt = 0
cntWrong = 0
for timestep, rewards_assigned_event in rewards_assigned_events.items():
    # back indexing rewards out from increase in pool_delegated_stake / 0.11 * 0.89
    if is_first:
        is_first = False
        pass
    else:       
        print(type(rewards_assigned_event))
        modeled_indexing_rewards = indexing_rewards_modeled[timestep]
        event_indexing_rewards = rewards_assigned_event[0]['amount']
        print(f"Event: Timestep={timestep-1}, Indexing Reward Tokens={modeled_indexing_rewards}")
        print(f"Model: Timestep={timestep-1}, Indexing Reward Tokens={indexing_rewards}")
        print("Tokens Within 1%?", 0.99 <= abs(stake_deposited_event[0]['tokens'] / indexing_rewards) <= 1.01)
        print()
        try:
            ratio = abs(modeled_indexing_rewards / indexing_rewards)
        except:
            ratio = np.inf
        exact = ratio == 1.0
        close = 0.99 <= ratio <= 1.01
        reallyClose = 0.99999 <= ratio <= 1.00001

        if exact:            
            cntExact += 1
        elif reallyClose:
            cntReallyClose += 1
        elif close:
            cntClose += 1
        else:
            cntWrong += 1
        cnt += 1
        print("Tokens Equal?", exact)
        print("Tokens Within 0.001%?", reallyClose)
        print("Tokens Within 1%?", close)
        print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   
        print()
print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")           


UNITTEST RESULTS
<class 'list'>
Event: Timestep=84, Indexing Reward Tokens=78681.945634908142661532
Model: Timestep=84, Indexing Reward Tokens=62945.55650792653160012713323
Tokens Equal? False
Tokens Within 1%? False

Shares Equal? False
Shares Within 0.001%? False
Shares Within 1%? False
Exact: 0, ReallyClose: 0, Close: 0, Wrong: 1, Total Number: 1

<class 'list'>
Event: Timestep=162, Indexing Reward Tokens=37810.500232191486929607
Model: Timestep=162, Indexing Reward Tokens=30248.40018575319793930318607
Tokens Equal? False
Tokens Within 1%? False

Shares Equal? False
Shares Within 0.001%? False
Shares Within 1%? False
Exact: 0, ReallyClose: 0, Close: 0, Wrong: 2, Total Number: 2

<class 'list'>
Event: Timestep=166, Indexing Reward Tokens=26381.732043325393053999
Model: Timestep=166, Indexing Reward Tokens=21105.38563466032030112046880
Tokens Equal? False
Tokens Within 1%? False

Shares Equal? False
Shares Within 0.001%? False
Shares Within 1%? False
Exact: 0, ReallyClose: 0, Close: 0

# Conclusion
The indexing reward token quantities are accurate to roughly 21 significant figures except in timestep 58.  